## Bonus exercise: match searches with bookings

Problems encountered: Difficulties to match variables in both files. It was not easy to define keys for merging. I decided to use the origin and final destination airport (IATA code), and the travel date. However, a large number of duplicates were encountered with these only 3 parameters. Duplicates were discarded to reduce the csv file size for the upload on github.

The analysis could be updated if matching parameters are provided (OfficeID, pos_oid, rloc?)

**Problem solved using pandas**

In [4]:
import pandas as pd

# Create a Dataframe object of the bookings.csv file
bookings = pd.read_csv('bookings.csv',sep='^',usecols={'dep_port','arr_port','brd_time           '})
bookings = bookings.rename(columns={'brd_time           ':'travel_date'})
bookings['travel_date'] = pd.to_datetime(bookings['travel_date'],errors='coerce')
bookings = bookings[pd.notnull(bookings['travel_date'])]

# Remove time variable. Only dates needed for comparisons with the travel dates set in searches.csv
bookings['travel_date'] = bookings['travel_date'].apply(pd.datetools.normalize_date) 

# Add a match column (used during merging)
bookings['match'] = 1

# Strip out the extra whitespaces in arr_port and dep_port
bookings["arr_port"] = bookings["arr_port"].map(str.strip)
bookings["dep_port"] = bookings["dep_port"].map(str.strip)

# Remove duplicates. Defined as rows with same travel_date, dep_port and arr_port
bookings=bookings.drop_duplicates(['travel_date','dep_port','arr_port'])

bookings.head(3)

,dep_port,arr_port,travel_date,match
0,ZRH,LHR,2013-03-07,1
1,SAL,CLT,2013-04-12,1
2,SAL,CLT,2013-07-15,1


In [5]:
# Create a Dataframe object of the searches.csv file.
searches = pd.read_csv('searches.csv',sep='^',dtype=object)
searches['Seg1Date'] = pd.to_datetime(searches['Seg1Date'],errors='coerce')
searches = searches[pd.notnull(searches['Seg1Date'])]

# Remove duplicates
searches=searches.drop_duplicates(['Seg1Date','Origin','Destination'])

searches.head(3)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC


In [9]:
# Merge both dataframes using the travel date and the origin and final destinations as keys parameters
searches_updated = pd.merge(searches, bookings, how='left',left_on=['Seg1Date','Origin','Destination'],\
                  right_on=['travel_date','dep_port','arr_port'])

# Fill the NaN in the match column with 0s (part of the requirment)
searches_updated['match']=searches_updated['match'].fillna(0)

searches_updated = searches_updated.drop(['dep_port','arr_port','travel_date'], 1)
searches_updated.head(3)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,match
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0


In [15]:
# write the updated searches file as .csv
searches_updated.to_csv('searches_updated.csv.bz2',index=False, compression='bz2')